In [21]:
import torch
import transformers
import tokenizers
from tokenizers import Tokenizer
import pprint
from transformers import PreTrainedTokenizerFast
from transformers import BertConfig, BertLMHeadModel
import numpy as np
import json
import pickle

In [22]:
def calculate_distance(n1, n2):
    """
    takes 2 numpy array
    n1, n2, each a vector embedding for a single code_string 
    """
    euclidean_dist=np.linalg.norm(n1-n2)
    #print(n1==n2)
    manhattan_dist=np.sum(np.abs(n1 - n2))
    cosine_sim=np.dot(n1,n2)/(np.linalg.norm(n1)*np.linalg.norm(n2))
    dot_product=np.dot(n1,n2)

    return euclidean_dist, manhattan_dist, cosine_sim, dot_product

In [23]:
edistances = []
cosSims = []
dotPros = []

In [24]:
tokenizer=Tokenizer.from_file('./tokenizer_nlp')
fast_tokenizer = PreTrainedTokenizerFast(tokenizer_object=tokenizer)
fast_tokenizer.mask_token='<mask>'
fast_tokenizer.pad_token='<pad>'
# Load the pre-trained BERT model and tokenizer
model = BertLMHeadModel.from_pretrained('../saved_model/non_ast_transformer/checkpoint-12400/',output_hidden_states=True)
        

In [25]:
# read scd-88 data
with open('../saved_data/all.jsonl','r') as test_file:
    test_list = list(test_file)

In [26]:
json.loads(test_list[1*130])['code']

'W,H=map(int,raw_input().strip().split(" "))\n\nif W*3 == H*4:\n\tprint "4:3"\nelse:\n\tprint "16:9"\n'

In [27]:
euclids=list()
cosines=list()
dots=list()
manhattans=list()

with torch.no_grad():
    
    for i in range(0,88,1):
        pairs=list()

        p_euclid=list()
        p_cosine=list()
        p_dot=list()
        p_manhattan=list()

        for p in range(2,102,1):
            clone1 = json.loads(test_list[i*130])['code']
            clone2 = json.loads(test_list[i*130+p])['code']

            pairs.append([clone1, clone2])

            #print(clone1)
            #print(clone2)
            #print('---------')

        # get embedding
        for code_pair in pairs:
            code1 = code_pair[0]
            code2 = code_pair[1]
        
            # Tokenize the input sentence
            id1 = fast_tokenizer.encode(code1, return_tensors='pt', max_length=512, truncation=True)
            id2 = fast_tokenizer.encode(code2, return_tensors='pt', max_length=512, truncation=True)
            #print(id1)
            #print(type(id1))
            #print(id1.shape)
            out_clone1 = model(id1)
            out_clone2 = model(id2)

            v1=torch.zeros(out_clone1.hidden_states[0].shape)# v1, v2 must match the shape
            v2=torch.zeros(out_clone2.hidden_states[0].shape)

            for i in range(len(out_clone1.hidden_states)):# we have 3 layers
                v1 += out_clone1.hidden_states[i]
            
            mean_embed_clone1 = torch.mean(v1, dim=1).squeeze()

            for i in range(len(out_clone2.hidden_states)):# we have 3 layers
                v2 += out_clone2.hidden_states[i]
            
            mean_embed_clone2 = torch.mean(v2, dim=1).squeeze()
            #print(mean_embed_clone2.shape)

            e,m,c,d = calculate_distance(mean_embed_clone1.numpy(), mean_embed_clone2.numpy())
            
            p_euclid.append(e)
            p_cosine.append(c)
            p_dot.append(d)
            p_manhattan.append(m)

        euclids.append(p_euclid)
        cosines.append(p_cosine)
        dots.append(p_dot)
        manhattans.append(p_manhattan)      



In [28]:
len(euclids)

88

In [17]:
with open('euclids_only_nlp.pkl', 'wb') as f:
    pickle.dump(euclids, f)

with open('cosine_only_nlp.pkl', 'wb') as f:
    pickle.dump(cosines, f)

with open('dot_only_nlp.pkl', 'wb') as f:
    pickle.dump(dots, f)

with open('manhattans_only_nlp.pkl', 'wb') as f:
    pickle.dump(manhattans, f)

In [14]:
#with open('./euclids_only_nlp.pkl', 'rb') as f:
#    l = pickle.load(f)


In [149]:
type(l)

list

In [151]:
l[50]

[1.8432447,
 5.644394,
 3.6613176,
 0.9989857,
 1.9702805,
 1.1753317,
 4.8816905,
 3.8191931,
 2.370223,
 1.3215207,
 1.9702805,
 2.4915447,
 4.537694,
 1.3256447,
 1.4553967,
 3.6063714,
 4.4988256,
 2.925626,
 2.517416,
 1.7358152,
 2.0121646,
 2.9400136,
 1.4553967,
 1.9702805,
 0.9852411,
 1.2771,
 2.4687014,
 2.1534142,
 1.8908473,
 2.1689742,
 1.9702805,
 1.3947265,
 2.6271634,
 1.3947265,
 1.6337475,
 2.2082965,
 3.038416,
 1.983991,
 3.7895055,
 2.7678428,
 4.6914897,
 1.4553967,
 2.3895223,
 2.1697364,
 2.9980059,
 4.0265017,
 1.9702805,
 2.925626,
 1.8635972,
 1.9702805,
 1.0534362,
 2.298209,
 1.5221684,
 2.6619682,
 2.292061,
 1.5770031,
 2.82202,
 2.331531,
 0.5164221,
 2.5341444,
 1.7862774,
 1.9749049,
 2.058806,
 2.2300646,
 2.5235548,
 2.341783,
 2.7148845,
 3.038416,
 1.5522405,
 1.2382998,
 1.8432447,
 1.7729185,
 2.2015765,
 1.9404409,
 3.0917122,
 2.1775548,
 1.7566792,
 1.2426649,
 1.7214543,
 2.3713617,
 1.9649094,
 3.282714,
 1.3947265,
 1.3947265,
 1.3947265,


In [8]:
len(euclids[0])

100